In [8]:

import pandas as pd

data = pd.read_csv(r"C:\Users\GOWRISHANKAR\car_sales_data.csv")
print(data.head())


  Manufacturer       Model  Engine size Fuel type  Year of manufacture  \
0         Ford      Fiesta          1.0    Petrol                 2002   
1      Porsche  718 Cayman          4.0    Petrol                 2016   
2         Ford      Mondeo          1.6    Diesel                 2014   
3       Toyota        RAV4          1.8    Hybrid                 1988   
4           VW        Polo          1.0    Petrol                 2006   

   Mileage  Price  
0   127300   3074  
1    57850  49704  
2    39190  24072  
3   210814   1705  
4   127869   4101  


In [9]:
print(data.columns)

Index(['Manufacturer', 'Model', 'Engine size', 'Fuel type',
       'Year of manufacture', 'Mileage', 'Price'],
      dtype='object')


In [10]:
print(data.shape)

(50000, 7)


In [11]:
print(data.tail)

<bound method NDFrame.tail of       Manufacturer       Model  Engine size Fuel type  Year of manufacture  \
0             Ford      Fiesta          1.0    Petrol                 2002   
1          Porsche  718 Cayman          4.0    Petrol                 2016   
2             Ford      Mondeo          1.6    Diesel                 2014   
3           Toyota        RAV4          1.8    Hybrid                 1988   
4               VW        Polo          1.0    Petrol                 2006   
...            ...         ...          ...       ...                  ...   
49995          BMW          M5          5.0    Petrol                 2018   
49996       Toyota       Prius          1.8    Hybrid                 2003   
49997         Ford      Mondeo          1.6    Diesel                 2022   
49998         Ford       Focus          1.0    Diesel                 2016   
49999           VW        Golf          1.4    Diesel                 2012   

       Mileage   Price  
0       

In [1]:
# Cell 1: load libraries and the cleaned CSV
import os
import pandas as pd
import numpy as np

# Update this path if your cleaned file is elsewhere
CLEANED_CSV = r"C:\Users\GOWRISHANKAR\car_sales_data_cleaned.csv"
OUT_DIR = r"C:\Users\GOWRISHANKAR\analysis_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

df = pd.read_csv(CLEANED_CSV)
print("Rows, cols:", df.shape)
df.head()



Rows, cols: (49988, 8)


,Manufacturer,Model,Engine size,Fuel type,Year of manufacture,Mileage,Price,Car Age
0,Ford,Fiesta,1.0,Petrol,2002,127300,3074,23
1,Porsche,718 Cayman,4.0,Petrol,2016,57850,49704,9
2,Ford,Mondeo,1.6,Diesel,2014,39190,24072,11
3,Toyota,Rav4,1.8,Hybrid,1988,210814,1705,37
4,Vw,Polo,1.0,Petrol,2006,127869,4101,19


In [4]:
# Cell 2: derived columns (safe calculations)
df = df.copy()

# Avoid division by zero / NaN
df['Mileage_safe'] = df['Mileage'].replace({0: np.nan})
df['Car Age'] = df.get('Car Age', np.nan)  # ensure exists

# Price per 1,000 miles (more interpretable than raw ratio)
df['Price_per_1000_miles'] = np.where(
    df['Mileage_safe'].notna() & (df['Mileage_safe'] > 0),
    df['Price'] / (df['Mileage_safe'] / 1000.0),
    np.nan
)

# Price per year of age (if Car Age is 0 -> NaN)
df['Price_per_Year'] = np.where(
    df['Car Age'].notna() & (df['Car Age'] > 0),
    df['Price'] / df['Car Age'],
    np.nan
)

# Engine class bucket (simple bins)
df['Engine_Class'] = pd.cut(
    df['Engine size'].fillna(0),
    bins=[-0.1, 1.2, 1.6, 2.5, 3.5, 10],
    labels=['<=1.2L','1.3-1.6L','1.7-2.5L','2.6-3.5L','3.6L+'],
    include_lowest=True
)

# Simple boolean flags
df['Is_Premium_Price'] = df['Price'] > 25000
df['Is_Very_Low_Price'] = df['Price'] < 1000
df['High_Mileage'] = df['Mileage'].fillna(0) > 150000
df['Old_Car'] = df['Car Age'].fillna(999) >= 15

# Decision column: business-rule driven (customize thresholds below)
PRICE_PREMIUM = 25000
PRICE_TOO_LOW = 1000
MILEAGE_HIGH = 150000
AGE_OLD = 15

def decide_row(r):
    # priority order: extreme cheap / premium / needs inspection / good value / keep
    if pd.isna(r['Price']):
        return 'Unknown'
    if r['Price'] <= PRICE_TOO_LOW:
        return 'Discard / Verify (Too cheap)'
    if r['Price'] >= PRICE_PREMIUM:
        return 'Premium — Market separately'
    # inspection rules
    if (not pd.isna(r['Mileage']) and r['Mileage'] >= MILEAGE_HIGH) or (not pd.isna(r['Car Age']) and r['Car Age'] >= AGE_OLD):
        return 'Needs Inspection'
    # value signal: high price per 1000 miles indicates relatively high price for low usage
    if not pd.isna(r['Price_per_1000_miles']) and r['Price_per_1000_miles'] > 500:  # tune this
        return 'High Price per Usage — Inspect'
    # default: keep / regular listing
    return 'List Normally'

df['Decision'] = df.apply(decide_row, axis=1)


In [5]:
# Cell 3: show summaries and save
print("Sample of new columns:")
display(df.loc[:, ['Price','Mileage','Car Age','Price_per_1000_miles','Price_per_Year','Engine_Class','Decision']].head(10))

# Summary counts for decision categories
print("\nDecision value counts:")
print(df['Decision'].value_counts(dropna=False))

# Save augmented dataframe
aug_path = os.path.join(OUT_DIR, "car_sales_augmented_with_decisions.csv")
df.to_csv(aug_path, index=False)
print("Saved augmented dataframe to:", aug_path)


Sample of new columns:


,Price,Mileage,Car Age,Price_per_1000_miles,Price_per_Year,Engine_Class,Decision
0,3074,127300,23,24.147683,133.652174,<=1.2L,Needs Inspection
1,49704,57850,9,859.187554,5522.666667,3.6L+,Premium — Market separately
2,24072,39190,11,614.238326,2188.363636,1.3-1.6L,High Price per Usage — Inspect
3,1705,210814,37,8.087698,46.081081,1.7-2.5L,Needs Inspection
4,4101,127869,19,32.071886,215.842105,<=1.2L,Needs Inspection
5,29204,33603,7,869.089069,4172.000000,1.3-1.6L,Premium — Market separately
6,14350,86686,15,165.539995,956.666667,1.7-2.5L,Needs Inspection
7,30297,30663,10,988.063790,3029.700000,1.3-1.6L,Premium — Market separately
8,9977,73470,13,135.796924,767.461538,<=1.2L,List Normally
9,1049,262514,33,3.995977,31.787879,1.7-2.5L,Needs Inspection



Decision value counts:
Decision
Needs Inspection                  30855
Premium — Market separately        8584
List Normally                      5731
Discard / Verify (Too cheap)       2940
High Price per Usage — Inspect     1878
Name: count, dtype: int64
Saved augmented dataframe to: C:\Users\GOWRISHANKAR\analysis_outputs\car_sales_augmented_with_decisions.csv


In [7]:
# Cell 1 — imports and file paths
import os
from pathlib import Path
import pandas as pd
import numpy as np
from typing import Dict, Any

# Configure paths — edit if needed
BASE_DIR = Path(r"C:\Users\GOWRISHANKAR")
CLEANED_CSV = BASE_DIR / "car_sales_data_cleaned.csv"
OUT_DIR = BASE_DIR / "analysis_outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Notebook-only settings
CURRENT_YEAR = 2025  # change if needed

print("CLEANED_CSV:", CLEANED_CSV)
assert CLEANED_CSV.exists(), f"File not found: {CLEANED_CSV}"



CLEANED_CSV: C:\Users\GOWRISHANKAR\car_sales_data_cleaned.csv


In [8]:
# Cell 2 — load dataframe and create derived columns + decision logic
df = pd.read_csv(CLEANED_CSV)
df = df.copy()  # avoid modifying original in-place

# Ensure numeric columns are numeric
numeric_cols = ['Price', 'Mileage', 'Engine size', 'Year of manufacture', 'Car Age']
for c in numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# If Car Age missing but Year present, compute it
if 'Car Age' not in df.columns or df['Car Age'].isna().all():
    if 'Year of manufacture' in df.columns:
        df['Car Age'] = CURRENT_YEAR - df['Year of manufacture']
    else:
        df['Car Age'] = np.nan

# Safe helpers
def safe_div(numerator, denominator):
    """Return numerator/denominator or NaN if denominator is zero/NaN."""
    return np.where((denominator != 0) & (~pd.isna(denominator)), numerator / denominator, np.nan)

# Derived numeric ratios
# Price per 1000 miles (interpretable; if Mileage=0 or NaN => NaN)
df['Price_per_1000_miles'] = safe_div(df['Price'], df['Mileage'] / 1000.0)

# Price per year of age (if Car Age <=0 => NaN)
df['Price_per_Year'] = safe_div(df['Price'], df['Car Age'])

# Engine class buckets (tune bins as needed)
engine_bins = [-0.1, 1.2, 1.6, 2.5, 3.5, 100]
engine_labels = ['<=1.2L', '1.3-1.6L', '1.7-2.5L', '2.6-3.5L', '3.6L+']
df['Engine_Class'] = pd.cut(df['Engine size'].fillna(-1), bins=engine_bins, labels=engine_labels, include_lowest=True)

# Boolean flags (separate columns for clarity)
THRESH = {
    "PRICE_PREMIUM": 25000,
    "PRICE_TOO_LOW": 1000,
    "MILEAGE_HIGH": 150000,
    "AGE_OLD": 15,
    "PRICE_PER_1000_HIGH": 500  # tune to market
}

df['Flag_Premium_Price'] = df['Price'] >= THRESH['PRICE_PREMIUM']
df['Flag_Too_Low_Price'] = df['Price'] <= THRESH['PRICE_TOO_LOW']
df['Flag_High_Mileage'] = df['Mileage'] >= THRESH['MILEAGE_HIGH']
df['Flag_Old_Car'] = df['Car Age'] >= THRESH['AGE_OLD']
df['Flag_High_Price_per_1000'] = df['Price_per_1000_miles'] >= THRESH['PRICE_PER_1000_HIGH']

# Decision flags as a dictionary column (useful for filtering / explainability)
def compute_flags(row) -> Dict[str, bool]:
    return {
        "Premium": bool(row['Flag_Premium_Price']),
        "TooLowPrice": bool(row['Flag_Too_Low_Price']),
        "HighMileage": bool(row['Flag_High_Mileage']),
        "OldCar": bool(row['Flag_Old_Car']),
        "HighPricePerUsage": bool(row['Flag_High_Price_per_1000'])
    }

df['Decision_Flags'] = df.apply(compute_flags, axis=1)

# Rule-based Decision column (priority order)
def decide(row) -> str:
    if pd.isna(row['Price']):
        return "Unknown"
    if row['Flag_Too_Low_Price']:
        return "Discard / Verify (Too cheap)"
    if row['Flag_Premium_Price']:
        return "Premium — Market separately"
    if row['Flag_High_Mileage'] or row['Flag_Old_Car']:
        return "Needs Inspection"
    if row['Flag_High_Price_per_1000']:
        return "High Price per Usage — Inspect"
    return "List Normally"

df['Decision'] = df.apply(decide, axis=1)

# Optional: expand Decision_Flags into separate columns for easy filtering (already have boolean columns above)
# e.g., Flag_NeedsInspection = HighMileage OR OldCar
df['Flag_NeedsInspection'] = df['Flag_High_Mileage'] | df['Flag_Old_Car']

# Show a few columns to verify
display_cols = ['Manufacturer','Model','Price','Mileage','Car Age','Engine size',
                'Engine_Class','Price_per_1000_miles','Price_per_Year','Decision','Decision_Flags']
existing_display_cols = [c for c in display_cols if c in df.columns]
df[existing_display_cols].head(10)


,Manufacturer,Model,Price,Mileage,Car Age,Engine size,Engine_Class,Price_per_1000_miles,Price_per_Year,Decision,Decision_Flags
0,Ford,Fiesta,3074,127300,23,1.0,<=1.2L,24.147683,133.652174,Needs Inspection,"{'Premium': False, 'TooLowPrice': False, 'High..."
1,Porsche,718 Cayman,49704,57850,9,4.0,3.6L+,859.187554,5522.666667,Premium — Market separately,"{'Premium': True, 'TooLowPrice': False, 'HighM..."
2,Ford,Mondeo,24072,39190,11,1.6,1.3-1.6L,614.238326,2188.363636,High Price per Usage — Inspect,"{'Premium': False, 'TooLowPrice': False, 'High..."
3,Toyota,Rav4,1705,210814,37,1.8,1.7-2.5L,8.087698,46.081081,Needs Inspection,"{'Premium': False, 'TooLowPrice': False, 'High..."
4,Vw,Polo,4101,127869,19,1.0,<=1.2L,32.071886,215.842105,Needs Inspection,"{'Premium': False, 'TooLowPrice': False, 'High..."
5,Ford,Focus,29204,33603,7,1.4,1.3-1.6L,869.089069,4172.000000,Premium — Market separately,"{'Premium': True, 'TooLowPrice': False, 'HighM..."
6,Ford,Mondeo,14350,86686,15,1.8,1.7-2.5L,165.539995,956.666667,Needs Inspection,"{'Premium': False, 'TooLowPrice': False, 'High..."
7,Toyota,Prius,30297,30663,10,1.4,1.3-1.6L,988.063790,3029.700000,Premium — Market separately,"{'Premium': True, 'TooLowPrice': False, 'HighM..."
8,Vw,Polo,9977,73470,13,1.2,<=1.2L,135.796924,767.461538,List Normally,"{'Premium': False, 'TooLowPrice': False, 'High..."
9,Ford,Focus,1049,262514,33,2.0,1.7-2.5L,3.995977,31.787879,Needs Inspection,"{'Premium': False, 'TooLowPrice': False, 'High..."


In [9]:
# Cell 3 — quick counts, diagnostics, and save augmented CSV
# Summary of decision counts
print("Decision value counts:")
print(df['Decision'].value_counts(dropna=False))
print("\nTop Decision flag combinations (sample):")
display(df['Decision_Flags'].value_counts().head(10))

# Save augmented dataframe
aug_path = OUT_DIR / "car_sales_augmented_with_decisions.csv"
df.to_csv(aug_path, index=False)
print("\nSaved augmented dataframe to:", aug_path)


Decision value counts:
Decision
Needs Inspection                  30855
Premium — Market separately        8584
List Normally                      5731
Discard / Verify (Too cheap)       2940
High Price per Usage — Inspect     1878
Name: count, dtype: int64

Top Decision flag combinations (sample):


Decision_Flags
{'Premium': False, 'TooLowPrice': False, 'HighMileage': False, 'OldCar': True, 'HighPricePerUsage': False}     19415
{'Premium': False, 'TooLowPrice': False, 'HighMileage': True, 'OldCar': True, 'HighPricePerUsage': False}      11140
{'Premium': True, 'TooLowPrice': False, 'HighMileage': False, 'OldCar': False, 'HighPricePerUsage': True}       7372
{'Premium': False, 'TooLowPrice': False, 'HighMileage': False, 'OldCar': False, 'HighPricePerUsage': False}     5731
{'Premium': False, 'TooLowPrice': True, 'HighMileage': True, 'OldCar': True, 'HighPricePerUsage': False}        2939
{'Premium': False, 'TooLowPrice': False, 'HighMileage': False, 'OldCar': False, 'HighPricePerUsage': True}      1878
{'Premium': True, 'TooLowPrice': False, 'HighMileage': False, 'OldCar': False, 'HighPricePerUsage': False}       544
{'Premium': True, 'TooLowPrice': False, 'HighMileage': False, 'OldCar': True, 'HighPricePerUsage': False}        376
{'Premium': False, 'TooLowPrice': False, 'HighMil


Saved augmented dataframe to: C:\Users\GOWRISHANKAR\analysis_outputs\car_sales_augmented_with_decisions.csv
